In [ ]:
#| default_exp py2pyi

# Create delegated pyi

## Setup

In [ ]:
#| export
import ast, sys, inspect, re, os, importlib.util, importlib.machinery

from ast import parse, unparse
from inspect import signature, getsource
from fastcore.utils import *
from fastcore.meta import delegates

In [ ]:
#| hide
import copy
from fastcore.test import *

## Basics

In [ ]:
#| export
def imp_mod(module_path, package=None):
    "Import dynamically the module referenced in `fn`"
    module_path = str(module_path)
    module_name = os.path.splitext(os.path.basename(module_path))[0]
    spec = importlib.machinery.ModuleSpec(module_name, None, origin=module_path)
    module = importlib.util.module_from_spec(spec)
    spec.loader = importlib.machinery.SourceFileLoader(module_name, module_path)
    if package is not None: module.__package__ = package
    module.__file__ = os.path.abspath(module_path)
    spec.loader.exec_module(module)
    return module

In [ ]:
fn = Path('test_py2pyi.py')

In [ ]:
mod = imp_mod(fn)
a = mod.A()
a.h()

1

In [ ]:
#| export
def _get_tree(mod):
    return parse(getsource(mod))

In [ ]:
tree = _get_tree(mod)

In [ ]:
#| export
@patch
def __repr__(self:ast.AST):
    return unparse(self)

@patch
def _repr_markdown_(self:ast.AST):
    return f"""```python
{self!r}
```"""

In [ ]:
# for o in enumerate(tree.body): print(o)

In [ ]:
node = tree.body[4]
node

```python
def f(a: int, b: str='a') -> str:
    """I am f"""
    return 1
```

In [ ]:
#| export
functypes = (ast.FunctionDef,ast.AsyncFunctionDef)

In [ ]:
isinstance(node, functypes)

True

In [ ]:
#| export
def _deco_id(d:Union[ast.Name,ast.Attribute])->bool:
    "Get the id for AST node `d`"
    return d.id if isinstance(d, ast.Name) else d.func.id

def has_deco(node:Union[ast.FunctionDef,ast.AsyncFunctionDef], name:str)->bool:
    "Check if a function node `node` has a decorator named `name`"
    return any(_deco_id(d)==name for d in getattr(node, 'decorator_list', []))

In [ ]:
nm = 'delegates'
has_deco(node, nm)

False

In [ ]:
node = tree.body[5]
node

```python
@delegates(f)
def g(c, d: X, **kwargs) -> str:
    """I am g"""
    return 2
```

In [ ]:
has_deco(node, nm)

True

## Function processing

In [ ]:
def _proc_body   (node, mod): print('_proc_body', type(node))
def _proc_func   (node, mod): print('_proc_func', type(node))
def _proc_class  (node, mod): print('_proc_class', type(node))
def _proc_patched(node, mod): print('_proc_patched', type(node))

In [ ]:
#| export
def _get_proc(node):
    if isinstance(node, ast.ClassDef): return _proc_class
    if not isinstance(node, functypes): return None
    if not has_deco(node, 'delegates'): return _proc_body
    if has_deco(node, 'patch'): return _proc_patched
    return _proc_func

In [ ]:
#| export
def _proc_tree(tree, mod):
    for node in tree.body:
        proc = _get_proc(node)
        if proc: proc(node, mod)

In [ ]:
#| export
def _clean_patched_node(node):
    "Clean the patched node in-place."
    # When moving a patched node to its parent, we no longer need the patch decorator and parent annotation.
    node.decorator_list = [deco for deco in node.decorator_list if getattr(deco, "id", None) != "patch"]
    node.args.args[0].annotation = None

In [ ]:
parent_node = copy.deepcopy(tree.body[7])
patched_node = copy.deepcopy(tree.body[10])
test_is(has_deco(patched_node, "patch"), True)
test_eq(str(patched_node.args.args[0].annotation), parent_node.name)

_clean_patched_node(patched_node)
test_is(has_deco(patched_node, "patch"), False)
test_eq(patched_node.args.args[0].annotation, None)

In [ ]:
#| export
def _is_empty_class(node):
    if not isinstance(node, ast.ClassDef): return False
    if len(node.body) != 1: return False
    child = node.body[0]
    if isinstance(child, ast.Pass): return True
    if isinstance(child, ast.Expr) and isinstance(child.value, (ast.Ellipsis, ast.Str)): return True
    return False

In [ ]:
empty_cls1, empty_cls2, empty_cls3 = ast.parse('''
class A: 
    """An empty class."""
class B: 
    pass
class C: 
    ...
''').body

test_is(_is_empty_class(empty_cls1), True)
test_is(_is_empty_class(empty_cls2), True)
test_is(_is_empty_class(empty_cls3), True)

non_empty_cls, empty_func = ast.parse('''
class A: 
    a = 1
def f():
    ...
''').body
test_is(_is_empty_class(non_empty_cls), False)
test_is(_is_empty_class(empty_func), False)

In [ ]:
#| export
def _add_patched_node_to_parent(node, parent):
    "Add a patched node to its parent."
    # if the patch node updates an existing class method, let's replace it.
    for i, child in enumerate(parent.body):
        if hasattr(child, "name") and child.name == node.name:
            parent.body[i] = node
            return
    # if we've made it this far the patched node must be new, so we append it to the parent's list of children
    if _is_empty_class(parent): parent.body = [node]
    else: parent.body.append(node)

In [ ]:
# we could have reused `parent_node` and `patched_node` from the previous cells.
# copying them here allows us to run this cell multiple times which makes it a little easier to write tests.

parent_node = copy.deepcopy(tree.body[7])
patched_node = copy.deepcopy(tree.body[11])
test_eq(len(parent_node.body),1)
_add_patched_node_to_parent(patched_node, parent_node)
test_eq(len(parent_node.body),2)
test_eq(parent_node.body[-1], patched_node)

# patched node replaces an existing class method (A.h)
patched_h_node = ast.parse("""
@patch
def h(self: A, *args, **kwargs):
    ...
""", mode='single').body[0]

_add_patched_node_to_parent(patched_h_node, parent_node)
test_eq(len(parent_node.body), 2)
test_eq(parent_node.body[0], patched_h_node)

# patched node is added to an empty class
empty_cls, patched_node = ast.parse('''
class Z: 
    """An empty class."""

@patch
def a(self: Z, *args, **kwargs):
    ...
''').body

test_eq(len(empty_cls.body), 1)
test_ne(empty_cls.body[0], patched_node)
_add_patched_node_to_parent(patched_node, empty_cls)
test_eq(len(empty_cls.body), 1)
test_eq(empty_cls.body[0], patched_node)

In [ ]:
#| export
def _proc_patches(tree, mod):
    "Move all patched methods to their parents."
    class_nodes = {}  # {class_name: position in node tree}
    i = 0
    while i < len(tree.body):
        node = tree.body[i]
        if isinstance(node, ast.ClassDef):
            # patched nodes can access their parent using different scopes:
            #  - local scope (e.g. self: A)
            #  - modular scope (e.g. self: example.A) [used by @delegates]
            class_nodes.update({node.name: i, f"{mod.__name__}.{node.name}": i})
        elif isinstance(node, ast.FunctionDef) and has_deco(node, "patch"):
            annotation = node.args.args[0].annotation
            # a patched node can have 1 or more parents
            parents = annotation.elts if hasattr(annotation, "elts") else [annotation]
            parents = list(map(str, parents))
            parents_in_mod = [parent for parent in parents if parent in class_nodes]
            # we can move the patched node if at least one parent lives in the current module
            if parents_in_mod:
                _clean_patched_node(node)
                for parent in parents_in_mod:
                    parent_node = tree.body[class_nodes[parent]]
                    _add_patched_node_to_parent(node, parent_node)
                tree.body.pop(i)
                i -= 1  # as we've removed the patched node from the tree we need to decrement the loop counter
        i += 1

In [ ]:
#| export
def _proc_mod(mod):
    tree = _get_tree(mod)
    _proc_tree(tree, mod)
    _proc_patches(tree, mod)
    return tree

In [ ]:
raw_tree = _get_tree(mod)
processed_tree = _proc_mod(mod)
n_raw_tree_nodes = len(raw_tree.body)
# mod contains 3 patch methods so our processed_tree should have 3 less nodes 
test_eq(len(processed_tree.body), n_raw_tree_nodes-3)

_proc_class <class 'ast.ClassDef'>
_proc_body <class 'ast.FunctionDef'>
_proc_func <class 'ast.FunctionDef'>
_proc_body <class 'ast.FunctionDef'>
_proc_class <class 'ast.ClassDef'>
_proc_class <class 'ast.ClassDef'>
_proc_patched <class 'ast.FunctionDef'>
_proc_patched <class 'ast.FunctionDef'>
_proc_body <class 'ast.FunctionDef'>


In [ ]:
_proc_mod(mod);

_proc_class <class 'ast.ClassDef'>
_proc_body <class 'ast.FunctionDef'>
_proc_func <class 'ast.FunctionDef'>
_proc_body <class 'ast.FunctionDef'>
_proc_class <class 'ast.ClassDef'>
_proc_class <class 'ast.ClassDef'>
_proc_patched <class 'ast.FunctionDef'>
_proc_patched <class 'ast.FunctionDef'>
_proc_body <class 'ast.FunctionDef'>


In [ ]:
node.name

'g'

In [ ]:
sym = getattr(mod, node.name)
sym

<function test_py2pyi.g(c, d: test_py2pyi.X, *, b: str = 'a') -> str>

In [ ]:
sig = signature(sym)
print(sig)

(c, d: test_py2pyi.X, *, b: str = 'a') -> str


In [ ]:
#| export
def sig2str(sig):
    s = str(sig)
    s = re.sub(r"<class '(.*?)'>", r'\1', s)
    s = re.sub(r"dynamic_module\.", "", s)
    return s

In [ ]:
#| export
def ast_args(func):
    sig = signature(func)
    return ast.parse(f"def _{sig2str(sig)}: ...").body[0].args

In [ ]:
newargs = ast_args(sym)
newargs

```python
c, d: test_py2pyi.X, *, b: str='a'
```

In [ ]:
node.args

```python
c, d: X, **kwargs
```

In [ ]:
node.args = newargs
node

```python
@delegates(f)
def g(c, d: test_py2pyi.X, *, b: str='a') -> str:
    """I am g"""
    return 2
```

In [ ]:
#| export
def _body_ellip(n: ast.AST):
    stidx = 1 if isinstance(n.body[0], ast.Expr) and isinstance(n.body[0].value, ast.Str) else 0
    n.body[stidx:] = [ast.Expr(ast.Constant(...))]

In [ ]:
_body_ellip(node)
node

```python
@delegates(f)
def g(c, d: test_py2pyi.X, *, b: str='a') -> str:
    """I am g"""
    ...
```

In [ ]:
#| export
def _update_func(node, sym):
    """Replace the parameter list of the source code of a function `f` with a different signature.
    Replace the body of the function with just `pass`, and remove any decorators named 'delegates'"""
    # sym_args contains the complete set of node args including any delegates kwargs.
    # when adding the delegates kwargs any annotation with a user-defined type is given a fully qualified name (e.g. mod.submod.func).
    # unfortunately the fully qualified names don't match the import statements in the node tree.
    # this can break downstream applications such as "jump to definition" in IDEs.
    # to resolve this issue we replace the annotations of user-defined types with the original annotations.
    sym_args = ast_args(sym)
    node_args = {arg.arg: arg.annotation for arg in node.args.args + node.args.kwonlyargs}
    for arg in sym_args.args + sym_args.kwonlyargs:
        arg.annotation = node_args.get(arg.arg, arg.annotation)
    node.args = sym_args
    _body_ellip(node)
    node.decorator_list = [d for d in node.decorator_list if _deco_id(d) != 'delegates']

In [ ]:
tree = _get_tree(mod)
node = tree.body[5]
node

```python
@delegates(f)
def g(c, d: X, **kwargs) -> str:
    """I am g"""
    return 2
```

In [ ]:
_update_func(node, sym)
node

```python
def g(c, d: X, *, b: str='a') -> str:
    """I am g"""
    ...
```

In [ ]:
#| export
def _proc_body(node, mod): _body_ellip(node)

In [ ]:
#| export
def _proc_func(node, mod):
    sym = getattr(mod, node.name)
    _update_func(node, sym)

In [ ]:
tree = _proc_mod(mod)
tree.body[5]

_proc_class <class 'ast.ClassDef'>
_proc_class <class 'ast.ClassDef'>
_proc_class <class 'ast.ClassDef'>
_proc_patched <class 'ast.FunctionDef'>
_proc_patched <class 'ast.FunctionDef'>


```python
def g(c, d: X, *, b: str='a') -> str:
    """I am g"""
    ...
```

## Patch

In [ ]:
tree = _get_tree(mod)
node = tree.body[9]
node

```python
@patch
@delegates(j)
def k(self: (A, B), b: bool=False, **kwargs):
    return 1
```

In [ ]:
ann = node.args.args[0].annotation

In [ ]:
if hasattr(ann, 'elts'): ann = ann.elts[0]

In [ ]:
nm = ann.id
nm

'A'

In [ ]:
cls = getattr(mod, nm)
sym = getattr(cls, node.name)

In [ ]:
sig2str(signature(sym))

"(self: (test_py2pyi.A, test_py2pyi.B), b: bool = False, *, d: str = 'a')"

In [ ]:
_update_func(node, sym)

In [ ]:
node

```python
@patch
def k(self: (A, B), b: bool=False, *, d: str='a'):
    ...
```

In [ ]:
#| export
def _proc_patched(node, mod):
    ann = node.args.args[0].annotation
    if hasattr(ann, 'elts'): ann = ann.elts[0]
    cls = getattr(mod, ann.id)
    sym = getattr(cls, node.name)
    _update_func(node, sym)

In [ ]:
tree = _get_tree(mod)
tree.body[9]

```python
@patch
@delegates(j)
def k(self: (A, B), b: bool=False, **kwargs):
    return 1
```

## Class and file

In [ ]:
tree = _get_tree(mod)
node = tree.body[7]
node

```python
class A:

    @delegates(j)
    def h(self, b: bool=False, **kwargs):
        a = 1
        return a
```

In [ ]:
node.body

[@delegates(j)
 def h(self, b: bool=False, **kwargs):
     a = 1
     return a]

In [ ]:
#| export
def _proc_class(node, mod):
    cls = getattr(mod, node.name)
    _proc_tree(node, cls)

In [ ]:
tree = _proc_mod(mod)
tree.body[7]

```python
class A:

    def h(self, b: bool=False, *, d: str='a'):
        ...

    def k(self, b: bool=False, *, d: str='a'):
        ...

    def m(self, b: bool=False, *, d: str='a'):
        ...

    def n(self, b: bool=False, **kwargs):
        """No delegates here mmm'k?"""
        ...
```

In [ ]:
#| export
def create_pyi(fn, package=None):
    "Convert `fname.py` to `fname.pyi` by removing function bodies and expanding `delegates` kwargs"
    fn = Path(fn)
    mod = imp_mod(fn, package=package)
    tree = _proc_mod(mod)
    res = unparse(tree)
    fn.with_suffix('.pyi').write_text(res)

In [ ]:
create_pyi(fn)

In [ ]:
# fn = Path('/Users/jhoward/git/fastcore/fastcore/docments.py')
# create_pyi(fn, 'fastcore')

## Script

In [ ]:
#| export
from fastcore.script import call_parse

In [ ]:
#| export
@call_parse
def py2pyi(fname:str,  # The file name to convert
           package:str=None  # The parent package
          ):
    "Convert `fname.py` to `fname.pyi` by removing function bodies and expanding `delegates` kwargs"
    create_pyi(fname, package)

In [ ]:
#| export
@call_parse
def replace_wildcards(
    # Path to the Python file to process
    path: str):
    "Expand wildcard imports in the specified Python file."
    path = Path(path)
    path.write_text(expand_wildcards(path.read_text()))

# Export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()